# Klasyfikacja cyfr z użyciem MNIST i klasyfikatora SGD

W tym laboratorium zajmujemy się rozpoznawaniem cyfr na podstawie zbioru danych **MNIST**. Każda obserwacja to obrazek o wymiarach 28x28 pikseli przedstawiający pojedynczą cyfrę (0–9). Celem eksperymentu jest zbudowanie klasyfikatora, który będzie rozpoznawał, czy dana cyfra to **0** (klasyfikacja binarna), a następnie trenowanie klasyfikatora wieloklasowego.

Do budowy modelu wykorzystujemy:
- klasyfikator liniowy `SGDClassifier` z pakietu `sklearn`,
- podział danych na zbiór treningowy i testowy,
- walidację krzyżową,
- metryki takie jak dokładność i macierz pomyłek,
- zapis wyników do plików `.pkl`.

Dodatkowo, wizualizujemy wybrane dane wejściowe oraz porównujemy dokładność klasyfikacji na zbiorach treningowym i testowym.


## Import i wczytanie danych

In [ ]:

from sklearn.datasets import fetch_openml
import numpy as np
from matplotlib import pyplot as plt

mnist = fetch_openml('mnist_784', version=1, as_frame=True)

# Podgląd jednej próbki
pixels = np.array(mnist.data.loc[42]).reshape(28, 28)
plt.imshow(pixels, cmap='gray')
plt.title("Przykład obrazu: Cyfra {}".format(mnist.target[42]))
plt.show()
print((pixels > 0).astype(int))


## Przygotowanie danych

In [ ]:

X = mnist.data
y = mnist.target.astype(int)

# Sortowanie wg etykiety (dla eksperymentu)
y = y.sort_values(ascending=True)
X = X.reindex(y.index)

# Podział zbioru
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Konwersja do klasyfikacji binarnej: czy cyfra to 0?
y_train_0 = (y_train == 0).astype(int)
y_test_0 = (y_test == 0).astype(int)


## Trenowanie klasyfikatora SGD (klasyfikacja 0 vs reszta)

In [ ]:

from sklearn.linear_model import SGDClassifier
import time

start = time.time()
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_0)
print("Czas trenowania:", time.time() - start)

# Predykcje
y_train_0_pred = sgd_clf.predict(X_train)
y_test_0_pred = sgd_clf.predict(X_test)

# Dokładność
acc_train = sum(y_train_0 == y_train_0_pred) / len(y_train_0)
acc_test = sum(y_test_0 == y_test_0_pred) / len(y_test_0)
print("Dokładność (train):", acc_train)
print("Dokładność (test):", acc_test)

# Zapis do pliku
import pickle
with open("sgd_acc.pkl", "wb") as file:
    pickle.dump([acc_train, acc_test], file)


## Walidacja krzyżowa

In [ ]:

from sklearn.model_selection import cross_val_score

start = time.time()
score = cross_val_score(sgd_clf, X_train, y_train_0, cv=3, scoring="accuracy", n_jobs=-1)
print("Czas walidacji:", time.time() - start)
print("Wyniki walidacji:", score)

with open("sgd_cva.pkl", "wb") as f:
    pickle.dump(score, f)


## Klasyfikacja wieloklasowa i macierz pomyłek

In [ ]:

sgd_m_clf = SGDClassifier(random_state=42, n_jobs=-1)
sgd_m_clf.fit(X_train, y_train)

# Predykcje
print("Predykcja 10 pierwszych próbek:", sgd_m_clf.predict(mnist.data.iloc[:10]))

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

y_train_pred = cross_val_predict(sgd_m_clf, X_train, y_train, cv=3, n_jobs=-1)
conf_mx = confusion_matrix(y_train, y_train_pred)
print("Macierz pomyłek:
", conf_mx)

with open("sgd_cmx.pkl", "wb") as fi:
    pickle.dump(conf_mx, fi)
